In [1]:
import requests
import csv

from datetime import datetime
from google.auth import default
from google.auth.transport.requests import Request

In [2]:
def get_gcloud_credential():
    credentials, project = default()
    if not credentials.valid:
        if credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
    return credentials

In [3]:
def square_search(credentials, sw_coord, ne_coord):
    api_url = 'https://places.googleapis.com/v1/places:searchNearby'
    sw_lat, sw_long = [float(element) for element in sw_coord.split(',')]
    ne_lat, ne_long = [float(element) for element in ne_coord.split(',')]
    field = ('places.displayName,'
             'places.types,'
             'places.priceLevel,'
             'places.rating')
    
    header = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {credentials.token}',
        'X-Goog-FieldMask': field,
        'X-Goog-User-Project': credentials.quota_project_id
    }
    
    payload = {
        'locationRestriction': {
            'rectangle': {
                'southwest': {
                    'latitude': sw_lat,
                    'longitude': sw_long
                },
                'northeast': {
                    'latitude': ne_lat,
                    'longitude': ne_long
                },
            }
        }
    }
    
    response = requests.post(api_url, headers=header, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print(f'Error: {response.status_code}')
        print(response.json())
        return None

In [4]:
def dataset_maker(data):
    now = datetime.now().strftime('%d-%m-%y_%H%M')
    banned_types = ['point_of_interest', 'establishment']
    file_path = f'dataset-output/square_{now}.csv'
    dataset = []
    
    for item in data.get('places', []):
        types = [t for t in item.get('types', []) if t not in banned_types]
        row = {
            'Name': item.get('displayName', {}).get('text', ),
            'Types': ', '.join(types),
            'Price Level': item.get('priceLevel', ),
            'Rating': item.get('rating', )
        }
        dataset.append(row)
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=dataset[0].keys())
        writer.writeheader()
        writer.writerows(dataset)
    
    print(f'Dataset saved to {file_path}')

In [7]:
credential = get_gcloud_credential()

sw = '-8.434478442624776, 115.0325277037326'
ne = '-8.429957597856815, 115.03706806410705'

places = square_search(credential, sw, ne)

if places:
    for place in places.get('places', []):
        display_name = place.get('displayName', 'no displayname found')
        print(f'Name        : {display_name.get("text", "name not found")}')
        print(f'Types       : {place.get("types", "types not found")}')
        print(f'Price Level : {place.get("priceLevel", "no price level found")}')
        print(f'Rating      : {place.get("rating", "rating not found")}')
        print('---------------------------------------------------------')
else:
    print('No places found')

dataset_maker(places)

Error: 400
{'error': {'code': 400, 'message': 'Location restriction unspecified.', 'status': 'INVALID_ARGUMENT'}}
No places found


AttributeError: 'NoneType' object has no attribute 'get'